# Advanced Hunting With Jupyter Notebooks

 ### Let's setup the connection first with the Graph Security API and request a token

In [217]:
import json
import pandas
import urllib.request
import urllib.parse

# Your MTP environment

tenantId = '00000000-0000-0000-0000-000000000000' # Replace with your Tenant ID
appId = '000000000-0000-0000-0000-000000000000' # Replace with your Application ID
appSecret = '0000000000000000000000000000000' # Replace with the Secret for your Application

url = "https://login.windows.net/%s/oauth2/token" % (tenantId)

resourceAppIdUri = 'https://api.security.microsoft.com' # Hello, MTP

body = {
    'resource' : resourceAppIdUri,
    'client_id' : appId,
    'client_secret' : appSecret,
    'grant_type' : 'client_credentials'
}

data = urllib.parse.urlencode(body).encode("utf-8")

req = urllib.request.Request(url, data)
response = urllib.request.urlopen(req)
jsonResponse = json.loads(response.read())
aadToken = jsonResponse["access_token"] # Access token for the next hour

### With KQL we payed close attention to a specific host 'koostest-srv-01'

In [218]:
# Specify your Advanced Hunting query (KQL)

query = ('''
DeviceNetworkEvents
| where Timestamp > ago(7d)
| where DeviceName hasprefix "koostest-srv-01"
| where RemoteIPType == "Public"
| where RemoteIP matches regex @'^[0-9]{1,3}.[0-9]{1,3}.[0-9]{1,3}.[0-9]{1,3}'
| project Timestamp, ActionType, RemoteIP, RemotePort, RemoteUrl, Protocol, InitiatingProcessFileName
''') # KQL

#### We specifcally connected to the Defender for Endpoint hunting API

In [219]:
url = "https://api.security.microsoft.com/api/advancedhunting/run" # Query the MTP Advanced Hunting API
headers = { 
	'Content-Type' : 'application/json',
	'Accept' : 'application/json',
	'Authorization' : "Bearer " + aadToken
}

data = json.dumps({ 'Query' : query }).encode("utf-8")

req = urllib.request.Request(url, data, headers)
response = urllib.request.urlopen(req)
jsonResponse = json.loads(response.read())
schema = jsonResponse["Schema"]
results = jsonResponse["Results"] # JSON response will be loaded in variable called 'results'

#### We used 'pandas' to normalize json output

pandas is a fast, powerful, flexible and easy to use open source data analysis and manipulation tool,
built on top of the Python programming language.

In [220]:
from pandas.io.json import json_normalize

print(" ")
print("Outbound connections from server 'koostest-srv-01.cybertron.com':")
print(" ")

pandas.json_normalize(results) # Convert JSON to a dataframe

 
Outbound connections from server 'koostest-srv-01.cybertron.com':
 


,Timestamp,ActionType,RemoteIP,RemotePort,RemoteUrl,Protocol,InitiatingProcessFileName
0,2020-12-10T21:06:55.1513236Z,ConnectionSuccess,20.54.213.113,443,winatp-gw-weu.microsoft.com,Tcp,
1,2020-12-10T21:07:29.2345823Z,ConnectionSuccess,52.114.159.32,443,v10c.events.data.microsoft.com,Tcp,svchost.exe
2,2020-12-10T21:10:26.8354221Z,ConnectionSuccess,104.17.50.74,443,api.nordvpn.com,Tcp,NordVPNSetup.tmp
3,2020-12-10T21:10:27.3700476Z,ConnectionSuccess,104.16.160.101,443,applytics.zwyr157wwiu6eior.com,Tcp,NordVPNSetup.tmp
4,2020-12-10T21:10:27.7985429Z,ConnectionSuccess,69.192.193.125,80,go.microsoft.com,Tcp,NordVPNTapSetup.exe
...,...,...,...,...,...,...,...
583,2020-12-10T21:25:55.8565365Z,ConnectionSuccess,40.126.0.70,443,login.microsoftonline.com,Tcp,backgroundTaskHost.exe
584,2020-12-10T21:25:55.9494487Z,ConnectionSuccess,72.21.91.29,80,ocsp.digicert.com,Tcp,backgroundTaskHost.exe
585,2020-12-10T20:27:07.9475748Z,ConnectionSuccess,20.190.133.78,443,login.microsoftonline.com,Tcp,backgroundTaskHost.exe
586,2020-12-10T20:53:32.3183827Z,ConnectionSuccess,104.17.49.74,443,api.nordvpn.com,Tcp,NordVPNSetup.tmp


### Now let's install the goodness from MS Threat Intelligence Center (MSTIC)

Use MSTICPY for instance to find IP Geo location. This can be used to plot it on a map or calculate distances.

In [221]:
!pip install msticpy -q 

from IPython.display import display
import pandas as pd

import msticpy.sectools as sectools
from msticpy.nbtools import *
from msticpy.nbtools.entityschema import IpAddress, GeoLocation
from msticpy.sectools.geoip import GeoLiteLookup, IPStackLookup

#### We used 'folium' to add markers to a map and draw the map 

In [222]:
import folium

map = folium.Map()

for result in results:
    ip_address = result['RemoteIP']
    process = result["InitiatingProcessFileName"]

    iplocation = GeoLiteLookup(api_key="0000000000000000")
    loc_result, ip_entity = iplocation.lookup_ip(ip_address)
    
    data = pandas.json_normalize(loc_result)

    lat = data.get('location.latitude', 0)
    lon = data.get('location.longitude', 0)

    marker = folium.Marker(location=[lat,lon],popup=process,icon=folium.Icon(color='red')) # Add red marker for every connection
    marker.add_to(map)
    
marker = folium.Marker(location=[52.3702,4.8951],popup='You are here',icon=folium.Icon(color='green')) # Amsterdam
marker.add_to(map)

map

### Let's use Graphistry to build a hypergraph and use our cognitive skills to interpret the data

We need to install the module and input our authorization key

In [223]:
!pip install graphistry -q 

%reload_ext graphistry
import graphistry

print(graphistry.__version__)

graphistryKey = "000000000000000000000000000000000000000000000000000000000000000"

graphistry.register(key=graphistryKey)

0.14.1


In [224]:
BigPileOfData = pandas.json_normalize(results)

hg = graphistry.hypergraph(BigPileOfData,["RemoteIP", "RemoteUrl", "InitiatingProcessFileName"],direct=True,opts={'EDGES': {
        'InitiatingProcessFileName': ['RemoteUrl'], 
        'RemoteUrl': ['RemoteIP']
}}
)

hg['graph'].plot() # show me the money :-)

# hg['graph'].plot(render=False) // direct url to the graph

# links 1176
# events 588
# attrib entities 743
